# Data acquisition

In this section, we will be using pandas to read the dataset in csv file format by using it's function read_csv() and we will be checking some properties of our dataframe.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# raw csv file collected from GitHub : @BDBC-KG-NLP/COVID-19-tracker
chinese_province_wise_data = {
    'sichuan'  : 'https://bit.ly/3fOW1BM',
    'anhui'    : 'https://bit.ly/2A7O2Ra',
    'shandong' : 'https://bit.ly/3g1Fo5J',
    'shanxi'   : 'https://bit.ly/3fWyCOJ',
    'jiangsu'  : 'https://bit.ly/2BHdCgk',
    'henan'    : 'https://bit.ly/2BDOhng',
    'zhejiang' : 'https://bit.ly/2NGddNX',
    'hainan'   : 'https://bit.ly/382c7oX',
    'hunan'    : 'https://bit.ly/31cDHyt',
    'guizhou'  : 'https://bit.ly/2Zn6T3l',
    'chongqing': 'https://bit.ly/2YwmvSy'
}

# list of provinces parsing from dict.keys()
provinces = chinese_province_wise_data.keys()

In [ ]:
# list to collect dataframes from each dataset
list_of_frames = []

# iterate and append frame in list_of_frames
for each_province in provinces:
  frame = pd.read_csv(chinese_province_wise_data[each_province], index_col=None, header=None)
  list_of_frames.append(frame)

# df : completely merged frame
df = pd.concat(list_of_frames, axis=0, ignore_index=True)

In [ ]:
# refactoring index bugs
df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
df.drop(df.index[[0]], inplace=True)

# refactor row index
df.reset_index(drop=True, inplace=True)

# custom header
headers = ['Gender', 'Age', 'Work', 'Diagnosis Time', 'Hubei Contact', 'Case Number', 'Permanent Residence', 'Hubei Contact Description', 'Case Announcement Province', 'Case Announcement City', 'Time', 'Route Location', 'Event', 'Description', 'Transportation', 'Track Number', 'Number Of Tracks']

# set custom header to dataframe
df.columns = headers
df.head(3)

,Gender,Age,Work,Diagnosis Time,Hubei Contact,Case Number,Permanent Residence,Hubei Contact Description,Case Announcement Province,Case Announcement City,Time,Route Location,Event,Description,Transportation,Track Number,Number Of Tracks
0,女,58,NaN,NaN,未知,NaN,NaN,NaN,四川省,泸州市,1月26日,"石河中心卫生院(107.328107,30.870779)",探望病人,"1月26日, 10：00，到石河中心卫生院探望病人",NaN,轨迹1,4
1,女,58,NaN,NaN,未知,NaN,NaN,NaN,四川省,泸州市,1月27日,"石河镇宝华村(107.327606,30.895882)",看望病危父亲,"1月27日, 16：00，到石河镇宝华村3组看望病危父亲",NaN,轨迹2,4
2,女,58,NaN,NaN,未知,NaN,NaN,NaN,四川省,泸州市,1月31日,"石河镇宝华村(107.327606,30.895882)",办丧事,"1月31日-2月5日, 在石河镇宝华村办丧事",NaN,轨迹3,4


In [ ]:
# check the shape of 
df.shape

(10790, 17)

# Data wrangling

In this section, we will be wrangling our data frame for the purpose of converting data from the initial format to a format that may be better for analysis. Sometimes, data-wrangling is referred to as **'data-preprocessing'**. In the beginning of processing, we will be identifying columns needed to be translated from **Chinese to English**.

Before, translating we need to drop some columns that are mostly NaN.



In [ ]:
# find columns NaN percentage
def check_nan() : 
  percent_nan = df.isnull().sum() * 100 / len(df)
  missing_value_in_df = pd.DataFrame(
      {
          'percent_nan': round(percent_nan, 2)
      }
  )
  print(missing_value_in_df)


check_nan()

                            percent_nan
Gender                             0.00
Age                                8.02
Work                              96.60
Diagnosis Time                    72.91
Hubei Contact                      0.10
Case Number                       99.66
Permanent Residence               18.84
Hubei Contact Description         85.39
Case Announcement Province         0.28
Case Announcement City             0.77
Time                               0.07
Route Location                    34.25
Event                             45.76
Description                        0.01
Transportation                    69.94
Track Number                       0.00
Number Of Tracks                   0.00


### Drop columns

By having an insight over NaN percentage on columns, we can see there are several columns that are mostly NaN. These columns are needed to be removed. But the question is how much NaN should we consider? According to some standard methodologies, columns with more than 50% NaN are useless. So, regarding the standard we will be dropping columns with NaN percentage more than 50%. Additionally, we will be removing data that are extremely not necessary.

In [ ]:
columns_to_drop = ['Work', 'Diagnosis Time', 'Permanent Residence', 'Hubei Contact', 'Case Number', 'Case Announcement City', 'Time', 'Hubei Contact Description', 'Transportation', 'Track Number', 'Number Of Tracks', 'Route Location']

'''
  axis = 1 : for dropping column
  inplace = True : upgrading the dataframe inline
'''
df.drop(columns=columns_to_drop, axis=1, inplace=True)

# find columns NaN percentage
check_nan()

                            percent_nan
Gender                             0.00
Age                                8.02
Case Announcement Province         0.28
Event                             45.76
Description                        0.01


### Drop rows initially

We have so much rows in our dataframe. That's cool unless we think of our translator API performance which we're gonna use next. Translating more than 10K rows in a dataframe might take **10-30 minutes**. Sad, but true 🥺. We cannot define all the important features of our dataset before translation but we can initally detect some of the features that are must for our analysis. 

Initially, Age, Permanent Resident, Case Announcement Province, Announcement City and Time are so important feature that must not be NaN. So, we can think of dropping these.

In [ ]:
drop_nan_rows = ['Age', 'Case Announcement Province']
df.dropna(subset=drop_nan_rows, axis=0, inplace=True)

# reset index, because we droped two rows
df.reset_index(drop=True, inplace=True)

In [ ]:
# find columns NaN percentage
check_nan()

                            percent_nan
Gender                             0.00
Age                                0.00
Case Announcement Province         0.00
Event                             45.72
Description                        0.00


In [ ]:
df.shape

(9898, 5)

### Correcting data type

In [ ]:
df['Age'].unique()

array(['58', '51', '50', '23', '28', '70', '54', '27', '18', '53', '43',
       '46', '37', '56', '10', '79', '57', '61', '39', '36', '41', '76',
       '20', '66', '64', '30', '65', '77', '6', '71', '68', '88', '49',
       '85', '45', '21', '55', '62', '48', '63', '47', '24', '73', '72',
       '3', '12', '52', '22', '25', '19', '29', '4', '33', '38', '35',
       '78', '34', '31', '26', '75', '7', '42', '32', '44', '81', '80',
       '17', '13', '67', '60', '9', '59', '69', '74', '年龄', '40', '16',
       '11', '2', '90', '14', '15', '84', '89', '1', '5', '87', '86',
       '83', '93', '96', '82', '8', '91'], dtype=object)

In [ ]:
# Replacing chinese word with average age = 44.75 => 45
df["Age"].replace('年龄', '45', inplace=True)

In [ ]:
df[["Age"]] = df[["Age"]].astype("int")

In [ ]:
# check NaN and replace with custom string
# because translation api doesn't take NaN
df['Event'].replace(np.nan, '', inplace=True)
check_nan()

                            percent_nan
Gender                              0.0
Age                                 0.0
Case Announcement Province          0.0
Event                               0.0
Description                         0.0


In [ ]:
df.head()

,Gender,Age,Case Announcement Province,Event,Description
0,女,58,四川省,探望病人,"1月26日, 10：00，到石河中心卫生院探望病人"
1,女,58,四川省,看望病危父亲,"1月27日, 16：00，到石河镇宝华村3组看望病危父亲"
2,女,58,四川省,办丧事,"1月31日-2月5日, 在石河镇宝华村办丧事"
3,女,58,四川省,被送至集中医学观察点隔离,"2月9日, 20：00，因与大竹县第4例确诊病例冯某密接者被送至集中医学观察点隔离"
4,女,51,四川省,回家,"1月19日, 雷某英独自从武汉返回泸县牛滩镇寿尊村10社家中"


In [ ]:
# swap first two columns position for iloc translation
c = df.columns
df[[c[0], c[1]]] = df[[c[1], c[0]]]
df.head(3)

,Gender,Age,Case Announcement Province,Event,Description
0,58,女,四川省,探望病人,"1月26日, 10：00，到石河中心卫生院探望病人"
1,58,女,四川省,看望病危父亲,"1月27日, 16：00，到石河镇宝华村3组看望病危父亲"
2,58,女,四川省,办丧事,"1月31日-2月5日, 在石河镇宝华村办丧事"


In [ ]:
# reset column header
df = df.rename(columns={'Gender': 'Age', 'Age': 'Gender'})
df.head(3)

,Age,Gender,Case Announcement Province,Event,Description
0,58,女,四川省,探望病人,"1月26日, 10：00，到石河中心卫生院探望病人"
1,58,女,四川省,看望病危父亲,"1月27日, 16：00，到石河镇宝华村3组看望病危父亲"
2,58,女,四川省,办丧事,"1月31日-2月5日, 在石河镇宝华村办丧事"


In [ ]:
check_nan()

                            percent_nan
Age                                 0.0
Gender                              0.0
Case Announcement Province          0.0
Event                               0.0
Description                         0.0


In [ ]:
df.shape

(9898, 5)

In [ ]:
df.dtypes

Age                            int64
Gender                        object
Case Announcement Province    object
Event                         object
Description                   object
dtype: object

In [ ]:
# translate gender manually
df['Gender'].unique()

array(['女', '男', '性别'], dtype=object)

In [ ]:
# replace manually
df['Gender'].replace({'女': 'female', '男': 'male'}, inplace=True)
df['Gender'].unique()

array(['female', 'male', '性别'], dtype=object)

In [ ]:
# count invalid data
df.Gender.str.count("性别").sum()

10

In [ ]:
# drop the rows containing this invalid words
df = df[~df['Gender'].isin(['性别'])]
df['Gender'].unique()

array(['female', 'male'], dtype=object)

In [ ]:
df.shape

(9888, 5)

# Data Translation
In this section, we will be translating our data from Chinese to English. Except Age and Number of track, we need to translate every other columns.

In [ ]:
! pip install googletrans

In [ ]:
# import the library
import googletrans
from googletrans import Translator

In [ ]:
# detect language
translator = Translator()

languages = translator.detect(['石河中心卫生院', 'English', 'বাংলা'])
for lang in languages:
    print(lang.lang, lang.confidence)

zh-CN 1.0
en 1.0
bn 1.0


In [ ]:
# copy dataframe for translation
df_en = df.copy()
df_en.iloc[:, 2:5].dtypes

Case Announcement Province    object
Event                         object
Description                   object
dtype: object

In [ ]:
# perform translation : skip first and last column
translations = {}
i = 0

for column in df_en.iloc[:, 2:5].columns:
    # unique elements of the column to avoid translating same thing multiple times
    unique_elements = df_en[column].unique()
    print(len(unique_elements))
    for element in unique_elements:
        # add translation to the dictionary
        translations[element] = translator.translate(element).text
        i = i + 1
        print('t => {}'.format(i))



# debug translation completion
print('Done with translation')

In [ ]:
# check how many unique cells were translated
len(translations)

8598

In [ ]:
# modify all the terms of the data frame by using the previously created dictionary
df_en.replace(translations, inplace = True)

# check translation
df_en.head()

,Age,Gender,Case Announcement Province,Event,Description
0,58,female,Sichuan Province,visit patient,"On January 26, 10:00, visit the patient at Shi..."
1,58,female,Sichuan Province,Visit a critically ill father,"On January 27, 16:00, visit the critically ill..."
2,58,female,Sichuan Province,Funeral,"January 31-February 5, funeral in Baohua Villa..."
3,58,female,Sichuan Province,Was sent to a centralized medical observation ...,"On February 9, at 20:00, a close contact with ..."
4,51,female,Sichuan Province,Come back home,"1月19日, 雷某英独自从武汉返回泸县牛滩镇寿尊村10社家中"


In [ ]:
from google.colab import files

df_en.to_csv('covid-en-0.5.csv')
files.download('covid-en-0.5.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>